# 8. Analysis and Visualization of Results

This final notebook consolidates all the generated data to perform a comprehensive analysis of the MCI conversion prediction model. It replicates the analyses from the original `Results dataset.ipynb`, `Metrics(new).ipynb`, `Demographics.ipynb`, and `Time to conversion.ipynb`.

The key analyses include:

1.  **Data Loading and Merging**: Loads the Monte Carlo dropout results, demographic information, and time-to-conversion data, merging them into a single master DataFrame.
2.  **Performance Metrics**: Calculates and displays key performance metrics for the model, including:
    *   Area Under the ROC Curve (AUC).
    *   Accuracy, Sensitivity, and Specificity at the optimal threshold (determined by G-Mean).
    *   Precision-Recall Curve and Confusion Matrix.
3.  **Correlation Analysis**: Investigates the relationships between the model's predictions (and uncertainty) and various clinical/demographic variables using correlation matrices and scatter plots.
4.  **Time-to-Conversion Analysis**: Explores the model's performance in predicting conversion at different time intervals (e.g., 1-year, 2-year recall).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix, precision_recall_curve, ConfusionMatrixDisplay
from scipy.stats import pearsonr, spearmanr, kendalltau

### Load and Merge Datasets

In [ ]:
# Define paths
results_path = Path("./results/")
mc_results_file = results_path / "mci_mc_dropout_results.pkl"
demographics_file = Path("./original_notebooks/excels/processed_demog.xlsx")
conversion_file = Path("./original_notebooks/excels/conv_with_time.xlsx")
adnimerge_file = Path("./data/raw/ADNIMERGE.csv")  # For full demographic info

# Load data
mc_df = pd.read_pickle(mc_results_file)

# It is better practice to load the full ADNIMERGE file for demographic data
# This ensures all subjects from the experiment can be found.
print("Loading ADNIMERGE for demographic and clinical data...")
demog_df = pd.read_csv(adnimerge_file, low_memory=False)
# Get baseline data for each subject
demog_df_baseline = demog_df[demog_df['VISCODE'] == 'bl']

# Merge MC results with demographic data
# The subject ID in our results is the 'PTID' in ADNIMERGE
results_df = pd.merge(mc_df, demog_df_baseline, left_on='subject_id', right_on='PTID', how='left')
results_df.rename(columns={"label": "Conversion"}, inplace=True)

print(f"Merged DataFrame shape: {results_df.shape}")
print("\nMerged DataFrame Head:")
display(results_df.head())

# Check for any subjects that didn't merge
if results_df['PTID'].isnull().any():
    print("\nWarning: Some subjects from MC results were not found in ADNIMERGE baseline data.")
    print(results_df[results_df['PTID'].isnull()]['subject_id'].tolist())

### Model Performance Evaluation

In [ ]:
# --- Performance Metrics ---
y_true = results_df["Conversion"]
y_pred_prob = results_df["mc_mean"]

auc = roc_auc_score(y_true, y_pred_prob)
print(f"Overall AUC: {auc:.4f}")

# Find best threshold using G-Mean
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
gmeans = np.sqrt(tpr * (1 - fpr))
best_thresh_idx = np.argmax(gmeans)
best_thresh = thresholds[best_thresh_idx]
print(f"Best Threshold (G-Mean): {best_thresh:.4f}")

# Predictions based on best threshold
y_pred_class = (y_pred_prob >= best_thresh).astype(int)

accuracy = accuracy_score(y_true, y_pred_class)
print(f"Accuracy at Best Threshold: {accuracy:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred_class)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
ppv = tp / (tp + fp)
npv = tn / (tn + fn)
print(f"Sensitivity (Recall): {sensitivity:.4f}")
print(f"Specificity: {specificity:.4f}")
print(f"Positive Predictive Value (PPV): {ppv:.4f}")
print(f"Negative Predictive Value (NPV): {npv:.4f}")

### Visualizations

In [ ]:
# ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f"AUC = {auc:.2f}")
plt.plot([0, 1], [0, 1], 'r--', label='No Skill')
plt.scatter(fpr[best_thresh_idx], tpr[best_thresh_idx], marker='o', color='black', label='Best Threshold')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve for MCI Conversion Prediction")
plt.legend()
plt.show()

# Confusion Matrix Display
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['sMCI', 'pMCI'])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

### Correlation Analysis

In [ ]:
# Correlation between predictions/uncertainty and other variables
# Update column names to match ADNIMERGE
# e.g., 'Age' is 'AGE', 'MMSE_dem' is 'MMSE', etc.
# Check your ADNIMERGE.csv for the exact column names. I will use common ones.
cols_to_correlate = [
    "mc_mean", "mc_std", "AGE", "PTEDUCAT", 
    "APOE4", "MMSE", "MOCA", "ADAS13", "FAQ"
]
# Filter for columns that actually exist in the dataframe
existing_cols = [col for col in cols_to_correlate if col in results_df.columns]

correlation_df = results_df[existing_cols].dropna()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_df.corr(method='pearson'), annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Pearson Correlation Matrix of Predictions and Clinical Variables")
plt.show()

# Scatter plot of prediction vs. uncertainty
plt.figure(figsize=(8, 6))
sns.scatterplot(data=results_df, x="mc_mean", y="mc_std", hue="Conversion")
plt.title("Prediction Mean vs. Standard Deviation")
plt.xlabel("Mean Prediction (pMCI Probability)")
plt.ylabel("Prediction Standard Deviation (Uncertainty)")
plt.show()